# Microglia detection

Detects microglia using a multiscale laplacian of gaussian filter.

In [ ]:
import numpy as np
from igraph import *
import matplotlib.pyplot as plt
import os
import readimg
from skimage.draw import circle_perimeter
import natsort
import blob
import pickle
import scipy.ndimage as ndi
from scipy.spatial.distance import cdist
                               
def read_thresholds(fileName):
	fd = open(fileName, 'r')
	data = fd.readlines()
	fd.close()
	
	tDict = {}
	for lineIndex, line in enumerate(data):
		if line[0]=='#':
			expName = line[1:].strip()
			if expName in tDict:
				print('Error reading thresholds, experiment already in dictionary')
			else:
				tDict[expName] = {}
		else:
			splittedLine = line.strip().split('\t')
			index, stackName, bestC = splittedLine
			tDict[expName][stackName] = [int(index), float(bestC)]
			
	return tDict
				
def get_files(inFolder, batchName, condStr=''):

	forbiddenFolders = ['ROI', 'maximum_projection', 'equalized']

	if os.name=='nt':
		if inFolder[-1]!='\\':
			if inFolder[-1]=='/':
				properInFolder = inFolder[:-1] + '\\'
				inFolder = inFolder[:-1] + '/'
			else:
				properInFolder = inFolder + '\\'
				inFolder = inFolder + '/'
	else:
		properInFolder = inFolder

	foldersTree = Graph(directed=True)
	foldersTree.add_vertex(name=batchName, fullName=batchName+'/', isFile=0)
	nameMap = {batchName:0}
	dataTree = os.walk(properInFolder+batchName)
	k = 1
	for fullPfolder, folders, files in dataTree:
		if os.name=='nt':
			Pfolder = fullPfolder.split('\\')[-1]
		else:
			Pfolder = fullPfolder.split('/')[-1]
		if Pfolder not in forbiddenFolders:
			sortedFiles = natsort.natsorted(files)
			sortedFolders = natsort.natsorted(folders)
			for file in sortedFiles:
				if condStr in file:
					foldersTree.add_vertex(name=file, fullName=foldersTree.vs[nameMap[Pfolder]]['fullName']+file, isFile=1)
					nameMap[file] = k
					k += 1
					foldersTree.add_edge(nameMap[Pfolder], nameMap[file])
			for folder in sortedFolders:
				if folder not in forbiddenFolders:
					foldersTree.add_vertex(name=folder, fullName=foldersTree.vs[nameMap[Pfolder]]['fullName']+folder+'/', isFile=0)
					nameMap[folder] = k
					k += 1			
					foldersTree.add_edge(nameMap[Pfolder], nameMap[folder])

	files = foldersTree.vs.select(isFile_eq=1)['fullName']
	
	return foldersTree, files
							
def draw_blobs(blobs, img, rf=1, vmax=255):
    
    if img.ndim==3:
        img_max = np.max(img, axis=0)
    else:
        img_max = img.copy()
        
    img_max[img_max>vmax] = vmax
    img_max_norm = np.round(255.*img_max/vmax).astype(np.uint8)
    
    img_draw = np.zeros((img_max.shape[0], img_max.shape[1], 3), dtype=np.uint8)
    img_draw[:,:,0] = img_max_norm.copy()
    img_draw[:,:,1] = img_max_norm.copy()
    img_draw[:,:,2] = img_max_norm.copy()
    for blob in blobs:
        rr, cc = circle_perimeter(int(blob[1]), int(blob[2]), radius=int(rf*blob[3]*np.sqrt(2)), 
                                  shape=img_max.shape)
        img_draw[rr, cc, 0] = 255
        img_draw[rr, cc, 1] = 0
        img_draw[rr, cc, 2] = 0

    return img_draw
    
def get_blobs(img, min_sigma=1, max_sigma=9, num_sigma=5, thresholds=None, overlap = 0.2):    
    blobs, _, sigma_dim = blob.blob_log(img, min_sigma=min_sigma, max_sigma=max_sigma, num_sigma=num_sigma, 
                                        threshold=thresholds, overlap=overlap, log_scale=False)
    
    blobs_pru = blob._prune_blobs(blobs, overlap, sigma_dim=sigma_dim)
    
    return blobs
                                 
def make_folders(outFolder, batchName):

    dirName = f'{outFolder}Microglia/{batchName}'
    if os.path.isdir(dirName)==False:
        os.mkdir(dirName)
    if os.path.isdir(f'{dirName}/images')==False:
        os.mkdir(f'{dirName}/images')


## Detect microglia in all images

In [ ]:
saveImgs = True	# Wheter to save intermediate steps and maximum projections

batchName = 'Vascular Project_Picture CD31-PDGFR-vGLUT2-IBA1'

inFolder = '/media/Data/cesar/Baptiste data/Project neuroinflammation/'
outFolder = '../data/'

shouldTestThresholds = False

tDict = read_thresholds('../data/thresholds.txt')

sigma = 5
threshold = 15

filesTree, files = get_files(inFolder, batchName, '.czi')
make_folders(outFolder, batchName)

q = 0 
all_blobs = {}
for q in range(len(files)):
    
    
    splitIndex = files[q].rfind('/')
    filePath, file = files[q][:splitIndex]+'/', files[q][splitIndex+1:]
    dotIndex = file.rfind('.')
    fileName = file[:dotIndex]


    img_vessel, scale = readimg.ReadImg(inFolder+filePath+file, 0)
    img_pericytes, scale = readimg.ReadImg(inFolder+filePath+file, 1)
    img_microglia, scale = readimg.ReadImg(inFolder+filePath+file, 2)

    max_img_vessel = np.max(img_vessel,axis=0)
    max_img_pericytes = np.max(img_pericytes,axis=0)
    max_img_microglia = np.max(img_microglia,axis=0)

    blobs = get_blobs(img_microglia, min_sigma=sigma, max_sigma=sigma, num_sigma=1, 
                         thresholds=[threshold])
    img_blobs = draw_blobs(blobs, img_microglia, 2, 128)
    
    outFolderComplete = f'{outFolder}Microglia/{batchName}/'
    plt.imsave(f'{outFolderComplete}images/{fileName}.png', img_blobs)
    all_blobs[fileName] = blobs
    pickle.dump(all_blobs, open(f'{outFolderComplete}blobs.dat', 'wb'))

## Measure detection quality using two hand-marked samples

In [ ]:
batchName = 'Vascular Project_Picture CD31-PDGFR-vGLUT2-IBA1'

inFolder = '/media/Data/cesar/Baptiste data/Project neuroinflammation/'
inFolderMarked = '/media/cesar/Dropbox/Dropbox/ufscar/Baptiste/Project neuroinflammation/data/Microglia/marked/binary/'
outFolder = '../data/'
markedImages = ['676-M-CD-AC-A', '697-M-HFD-AC-B']

sigma_arr = [3, 4, 5, 6, 7, 8]   # best: 5
thresh_arr = [5, 10, 15, 20, 25] # best: 15

R = 5

filesTree, files = get_files(inFolder, batchName, '.czi')
quality = {}
for q in range(len(markedImages)):
       
    fileName = markedImages[q]
    file = [file for file in files if fileName in file][0]
    splitIndex = file.rfind('/')
    filePath, file = file[:splitIndex]+'/', file[splitIndex+1:]
    dotIndex = file.rfind('.')

    quality[fileName] = {}
    
    img_microglia, scale = readimg.ReadImg(inFolder+filePath+file, 2)
    img_marked = plt.imread(f'{inFolderMarked}MAX_C3-{fileName}.png')[:,:,0]
    max_img_microglia = np.max(img_microglia,axis=0)
    
    img_lab, num_comps = ndi.label(img_marked)
    cm = ndi.center_of_mass(img_marked, img_lab, range(1, num_comps+1))
    
    for sigma in sigma_arr:

        quality[fileName][sigma] = {}
            
        for thresh in thresh_arr:
            blobs = get_blobs(img_microglia, min_sigma=sigma, max_sigma=sigma, num_sigma=1, 
                                 thresholds=[thresh])

            D = cdist(cm, blobs[:,1:-1])
            num_kp, num_cp = D.shape
            available = np.ones(num_cp)
            corresp = {}
            ind_kp_done = set()
            while True:
                ind = np.unravel_index(np.argmin(D), D.shape)
                ind_kp, ind_cp = ind

                if D[ind_kp, ind_cp]>R:
                    # No more cases to analyse
                    break
                D[ind_kp, ind_cp] = np.inf        
                if ind_kp not in ind_kp_done:
                    if available[ind_cp]==1:
                        ind_kp_done.add(ind_kp)
                        available[ind_cp] = 0
                        corresp[ind_kp] = ind_cp

            p = num_kp         # Num positive
            m = num_cp         # Num marked points
            tp = len(corresp)  # True positive
            fp = num_cp - tp   # False positive (detected but non-existent)
            fn = num_kp - tp   # False negative (not detected)

            
            quality[fileName][sigma][thresh] = {'p':p, 'm':m, 'tp':tp, 'fp':fp, 'fn':fn}
            pickle.dump(quality, open('quality.dat', 'wb'))
            

## Plot precision-recall curve

In [ ]:
prec_rec = {}
for q in range(len(markedImages)):
    fileName = markedImages[q]
    prec_rec[fileName] = {}
    for sigma in sigma_arr:
        prec_rec[fileName][sigma] = []
        for thresh in thresh_arr:
            res = quality[fileName][sigma][thresh]
            tp = res['tp']
            p = res['p']
            m = res['m']
            recall = tp/p
            precision = tp/m
            prec_rec[fileName][sigma].append([precision, recall])
            
        prec_rec[fileName][sigma] = np.array(prec_rec[fileName][sigma])
            
plt.figure()
c = ['b', 'r', 'g', 'y', 'm', 'c']
m = ['o', '^']
for q in range(len(markedImages)):
    fileName = markedImages[q]
    for sIdx, sigma in enumerate(sigma_arr):
        res = prec_rec[fileName][sigma]
        plt.scatter(res[:,0], res[:,1], marker=m[q], c=c[sIdx])
        plt.xlabel('Precision')
        plt.ylabel('Recall')
         